In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Model

# 1. Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 2. Normalize the data (scale pixel values to range [0, 1])
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 3. Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 4. Split the training data into training and validation sets (80% train, 20% validation)
validation_split = 0.2
num_train_samples = int((1 - validation_split) * x_train.shape[0])

x_train_final = x_train[:num_train_samples]
y_train_final = y_train[:num_train_samples]

x_val = x_train[num_train_samples:]
y_val = y_train[num_train_samples:]

# 5. Define image size for VGG16
IMG_SIZE = 224  # VGG16 expects 224x224 images

# 6. Function to preprocess and resize images
def preprocess_image(image, label):
    # Resize the image to the desired size
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

# 7. Create tf.data.Dataset objects for training, validation, and testing
batch_size = 32  # Adjust based on your GPU memory

# Training Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_final, y_train_final))
train_dataset = train_dataset.shuffle(buffer_size=10000)  # Shuffle the dataset
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)  # Resize images
train_dataset = train_dataset.batch(batch_size)  # Batch the data
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)  # Prefetch for performance

# Validation Dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

# Test Dataset
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

# 8. Load the pre-trained VGG16 feature extractor from Keras Applications
vgg16_base = VGG16(
    include_top=False,  # Exclude the top classification layers
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze the VGG16 base to prevent its weights from being updated during training
vgg16_base.trainable = False

# 9. Build the model using the Keras Functional API
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="images")
x = vgg16_base(inputs, training=False)  # Set training=False to avoid updating batch normalization layers
x = layers.Flatten()(x)  # Flatten the output of VGG16
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = Model(inputs=inputs, outputs=outputs)

# 10. Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# 11. Train the model using the tf.data.Dataset objects
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset
)

# 12. Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")

# 13. Generate predictions on a few test samples
print("Generate predictions for 3 samples")
for images, labels in test_dataset.take(1):
    predictions = model.predict(images[:3])
    print("Predictions shape:", predictions.shape)
    print(predictions)
    break
